# Personalize con evaluación temporal en el conjunto de reserva

Este cuaderno continúa en gran medida con el cuaderno básico, con la modificación adicional de retener el 10 % de los datos "futuros" para cada usuario. Entonces, configuramos un punto de conexión de inferencia para mover la recomendación y evaluar de forma externa los datos retenidos.

La otra pequeña diferencia es que predecimos vistas en vez de calificaciones. Nos parece más interesante demostrar la recomendación de películas poco populares (aunque muy personalizadas) que recomendar películas populares que todos disfrutarían (sin necesidades de personalización).

In [ ]:
import boto3, os
import json
import numpy as np
import pandas as pd
import time
from botocore.exceptions import ClientError
!pip install tqdm
from tqdm import tqdm_notebook
from metrics import mean_reciprocal_rank, ndcg_at_k, precision_at_k

In [ ]:
suffix = str(np.random.uniform())[4:9]

In [ ]:
bucket = "demo-temporal-holdout-"+   suffix        # replace with the name of your S3 bucket
filename = "DEMO-temporal-holdout.csv"

In [ ]:
!aws s3 mb s3://{bucket}

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Descargue y procese los datos de análisis comparativo

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -o ml-1m.zip
data = pd.read_csv('./ml-1m/ratings.dat', sep='::', names=['USER_ID','ITEM_ID','EVENT_VALUE', 'TIMESTAMP'])

In [ ]:
pd.set_option('display.max_rows', 5)

In [ ]:
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
print('unique users %d; unique items %d'%(
    len(data['USER_ID'].unique()), len(data['ITEM_ID'].unique())))
data

### Extraiga el 10 % de las últimas interacciones por usuario como prueba de reserva

In [ ]:
ranks = data.groupby('USER_ID').TIMESTAMP.rank(pct=True, method='first')
data = data.join((ranks>0.9).to_frame('holdout'))
holdout = data[data['holdout']].drop('holdout', axis=1)
data = data[~data['holdout']].drop('holdout', axis=1)

In [ ]:
print('unique users %d; unique items %d'%(
    len(data['USER_ID'].unique()), len(data['ITEM_ID'].unique())))
data

### Cargar los datos

In [ ]:
data.to_csv(filename, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

# Crear un esquema

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "DEMO-temporal-schema-"+suffix,
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

## Conjuntos de datos y grupos de conjuntos de datos

### Cree un grupo de conjuntos de datos.

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "DEMO-temporal-dataset-group-"+suffix
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

### Crear un tipo de conjunto de datos de 'interacciones'

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn,
    name = "DEMO-temporal-dataset-"+suffix
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Permisos del bucket de S3 para acceder a Personalize

### Adjuntar una política al bucket de S3

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy));

### Crear un rol con los permisos adecuados

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeS3Role-"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise
# sometimes need to wait a bit for the role to be created
time.sleep(45)
print(role_arn)

# Crear su trabajo de importación de conjunto de datos

Esta es la carga del conjunto de datos de interacciones

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "DEMO-temporal-dataset-import-job-"+suffix,
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

### Esperar a que el trabajo de importación del conjunto de datos y la ejecución del trabajo de importación de conjunto de datos tenga estado ACTIVO

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

# Crear la solución

In [ ]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

Hay varias recetas para diferentes situaciones. En este ejemplo, solo tenemos datos de interacciones, por lo que elegiremos una de las recetas básicas.

| ¿Es viable? | Receta | Descripción 
|-------- | -------- |:------------
| SÍ | aws-popularity-count | Calcula la popularidad de los elementos en función del recuento de eventos en relación con ese elemento en el conjunto de datos de interacciones entre el usuario y el elemento.
| SÍ | aws-hrnn | Predice los elementos con los que interactuará un usuario. Una red neuronal recurrente y jerárquica que puede modelar el orden temporal de las interacciones entre el usuario y el elemento.
| NO: requiere metadatos | aws-hrnn-metadata | Predice los elementos con los que interactuará un usuario. HRNN con características adicionales derivadas del contexto (metadatos de interacción entre el usuario y el elemento), metadatos del usuario (conjunto de datos del usuario) y metadatos del elemento (conjunto de datos del elemento).
| NO: para ladrones, requiere metadatos | aws-hrnn-coldstart | Predice los elementos con los que interactuará un usuario. Metadatos de HRNN con análisis personalizado de los elementos nuevos.
| NO: para consultas basadas en elementos | aws-sims | Calcula los elementos similares a un elemento determinado en función de la coocurrencia del elemento en el mismo historial del usuario en el conjunto de datos de la interacción entre el usuario y el elemento
| NO: para volver a clasificar una lista corta | aws-personalized-ranking | Vuelve a clasificar una lista de elementos para un usuario. Se entrena en el conjunto de datos de interacciones entre el usuario y el elemento. 


Nosotros (o autoML) podemos ejecutar todas estas recetas básicas y elegir el modelo de mejor rendimiento a partir de las métricas internas. Recomendamos hacer comparaciones, en especial con la base de referencia de popularidad, para ver las mejoras en las métricas a través de la personalización. Sin embargo, en esta demostración, elegiremos una receta: aws-hrnn, para centrarnos en las evaluaciones externas.

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

### Crear y esperar por una solución
Este proceso consta de 2 pasos
1. Cree la solución.
2. Cree una versión de la solución.

In [ ]:
create_solution_response = personalize.create_solution(
    name = "DEMO-temporal-solution-"+suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn,
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

### Esperar a que la versión de la solución esté ACTIVA

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

### Obtener las métricas de la solución

In [ ]:
get_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_metrics_response, indent=2))

# Crear y esperar una campaña

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "DEMO-temporal-campaign-"+suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 2,    
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

### Esperar a que la campaña tenga un estado ACTIVO

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

# Evaluar mediante métricas externas

En https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html, se ofrece una explicación acerca de las métricas de evaluación.

Por ejemplo, suponga que recomendamos cuatro elementos y dos de ellos son relevantes, $r=[0,1,0,1]$. En este caso, las métricas son las siguientes:

|Nombre	|Ejemplo	|Explicación
|:------|:----------|:----------
|Precision@K	|$\frac{2}{4} = 0.5$	|El total de los elementos relevantes dividido por el total de los elementos recomendados.
|Rangos recíprocos medios (MRR@K)	|${\rm mean}(\frac{1}{2} + \frac{1}{4}) = 0.375$	|Considera los efectos posicionales mediante el cálculo de la media de las posiciones inversas de todos los elementos relevantes.
|Ganancias acumuladas con descuento normalizadas (NDCG@K)	|$\frac{\frac{1}{\log(1 + 2)} + \frac{1}{\log(1 + 4)}}{\frac{1}{\log(1 + 1)} + \frac{1}{\log(1 + 2)}} = 0.65$	|Considera los efectos posicionales mediante la aplicación de pesos logarítmicos inversos basados ​​en las posiciones de los elementos relevantes, normalizados por las puntuaciones más altas posibles de las recomendaciones ideales.
|Precisión promedio (AP@K)	|${\rm mean}(\frac{1}{2} + \frac{2}{4}) = 0.5$	|Average precision@K donde K es la posición de cada elemento relevante.

Estas métricas difieren de las métricas internas en dos aspectos:
* Se evalúan en diferentes horarios, lo que puede implicar diferencias en las tasas de clics. Recomendamos mantener siempre las evaluaciones en el mismo horario para evitar desfases temporales.
* El ejemplo de las evaluaciones externas puede retener y considerar elementos múltiples como verdad fundamental, mientras que las evaluaciones internas solo retienen el último elemento en cada historial de usuario como verdad fundamental. No existe una preferencia absoluta acerca de cuantos elementos debería retener. Recomendamos diseñar los métodos de evaluación que son similares al caso de uso actual.

In [ ]:
relevance = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id)
    )
    rec_items = [int(x['itemId']) for x in rec_response['itemList']]
    relevance.append([int(x in true_items.values) for x in rec_items])

In [ ]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in relevance]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in relevance]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in relevance]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in relevance]))

### Opcional: los resultados son ligeramente mejores después de deduplicar los historiales de compras anteriores

In [ ]:
rel_dedup = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id)
    )
    past_items = data[data.USER_ID == user_id].ITEM_ID.values
    topk = [int(x['itemId']) for x in rec_response['itemList']]
    rec_items = [x for x in topk if x not in past_items]
    if len(rec_items) < 25:
        rec_items.extend([x for x in topk if x not in rec_items])
    rec_items = rec_items[:25]    

    rel_dedup.append([int(x in true_items.values) for x in rec_items])

In [ ]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in rel_dedup]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in rel_dedup]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in rel_dedup]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in rel_dedup]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in rel_dedup]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in rel_dedup]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in rel_dedup]))

### Intente comparar con la línea de base de popularidad como recomendador simulado

In [ ]:
topk = data.groupby("ITEM_ID").TIMESTAMP.count().sort_values(ascending=False).iloc[:100].index.values

In [ ]:
rel_popular = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_items = topk[:25]
    rel_popular.append([int(x in true_items.values) for x in rec_items])

In [ ]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in rel_popular]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in rel_popular]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in rel_popular]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in rel_popular]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in rel_popular]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in rel_popular]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in rel_popular]))

### Deduplicación de historiales de usuarios con la línea de base popularidad 

In [ ]:
rel_pop_dedup = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    past_items = data[data.USER_ID == user_id].ITEM_ID.values
    rec_items = [x for x in topk if x not in past_items]
    if len(rec_items) < 25:
        rec_items.extend([x for x in topk if x not in rec_items])
    rec_items = rec_items[:25]    
    rel_pop_dedup.append([int(x in true_items.values) for x in rec_items])

In [ ]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in rel_pop_dedup]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in rel_pop_dedup]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in rel_pop_dedup]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in rel_pop_dedup]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in rel_pop_dedup]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in rel_pop_dedup]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in rel_pop_dedup]))

# Limpiar

In [ ]:
personalize.delete_campaign(campaignArn=campaign_arn)
while len(personalize.list_campaigns(solutionArn=solution_arn)['campaigns']):
    time.sleep(5)

personalize.delete_solution(solutionArn=solution_arn)
while len(personalize.list_solutions(datasetGroupArn=dataset_group_arn)['solutions']):
    time.sleep(5)

for dataset in personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']:
    personalize.delete_dataset(datasetArn=dataset['datasetArn'])
while len(personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']):
    time.sleep(5)

personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)

# ¡Si utiliza un bucket personal, ejecute esta celda con precaución!

In [ ]:
!aws s3 rm s3://{bucket} --recursive